In [1]:
import os
import sys
import numpy as np

from Data_manager.Movielens.Movielens100KReader import Movielens100KReader
from Data_manager.AmazonReviewData.AmazonAllBeautyReader import AmazonAllBeautyReader 
from Data_manager.DataSplitter_global_timestamp import DataSplitter_global_timestamp

from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
# from Conferences.WWW.MultiVAE_our_interface.MultiVAE_RecommenderWrapper import (
# 	Mult_VAE_RecommenderWrapper,
# )
# from Conferences.KDD.CollaborativeDL_our_interface.CollaborativeDL_Matlab_RecommenderWrapper import (
# 	CollaborativeDL_Matlab_RecommenderWrapper,
# )
# from Conferences.KDD.CollaborativeVAE_our_interface.CollaborativeVAE_RecommenderWrapper import (
# 	CollaborativeVAE_RecommenderWrapper,
# )

from Base.Evaluation.Evaluator import EvaluatorHoldout


# from LIME_utils.perturb import perturb_rated_items, perturb_all_items

RECOMMENDERS = ["item-knn", "mult-vae"]
PERTURB_MODES = ['all-items', 'rated-items']

np.random.seed(42)

data_reader = Movielens100KReader(reload_from_original_data='always')
dataset = data_reader.load_data()
dataset.print_statistics()


save_folder = f"./tmp_DATA_{data_reader.DATASET_SUBFOLDER}"

# Create a training-validation-test split, for example by leave-1-out
# This splitter requires the DataReader object and the number of elements to holdout
dataSplitter = DataSplitter_global_timestamp(data_reader, k_out_percent=20, 
				use_validation_set=True, force_new_split=False)

# The load_data function will split the data and save it in the desired 
# folder. Once the split is saved, further calls to the load_data will 
# load the splitted data ensuring you always use the same split
dataSplitter.load_data(save_folder_path=save_folder)

# Get the three URMs and the ICMs (if present in the data Reader)
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

# ICM_dict = {'ICM_metadata': CSC sparse matrix n_items*n_feats }
ICM_dict = dataSplitter.get_loaded_ICM_dict()
ICM = ICM_dict['ICM_genre'] if 'ICM_genre' in ICM_dict else None

n_users, n_items = URM_train.shape


########## Recsys model part ##########

recommender_name = "item-knn"

if recommender_name == "item-knn":
	recommender = ItemKNNCFRecommender(URM_train)
	recommender.fit()
	
elif recommender_name == "mult-vae":
	recommender = Mult_VAE_RecommenderWrapper(URM_train)

	trained_model_path = './Conferences/WWW/MultiVAE_our_interface/trained_model/'

	if os.path.exists(trained_model_path):
		recommender.load_model(trained_model_path)
	else:
		recommender.fit(epochs=500)
		recommender.save_model(trained_model_path)

elif recommender_name == 'item-knn-cbf':
	recommender = ItemKNNCBFRecommender(URM_train, ICM)
	recommender.fit()

# elif recommender_name == "collaborative-vae":
# 	recommender = CollaborativeVAE_RecommenderWrapper(URM_train, ICM)
# 	recommender.fit()

evaluator = EvaluatorHoldout(URM_test, cutoff_list=[5])
results_run, results_run_string, raw_preds = evaluator.evaluateRecommender(recommender)
print(results_run_string)


Movielens100K: reload_from_original_data is 'always', previously loaded data will be ignored.
Movielens100K: Loading original data
Movielens100K: cleaning temporary files
Movielens100K: loading complete
Movielens100K: Verifying data consistency...
Movielens100K: Verifying data consistency... Passed!
Movielens100K: Found already existing folder '/home/saiprasad/Research/HomeDepot/HomeDepotEnsemble/reczilla/RecSys2019_DeepLearning_Evaluation/Data_manager/../Data_manager_split_datasets/Movielens100K/original/'
Movielens100K: Saving complete!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 1682
	Number of users: 943
	Number of interactions in URM_all: 100000
	Value range in URM_all: 1.00-5.00
	Interaction density: 6.30E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.06E+02
		 Max: 7.37E+02
	Interactions per item:
		 Min: 1.00E+00
		 Avg: 5.95E+01
		 Max: 5.83E+02
	Gini Index: 0.47

DataSplitter_global_timestamp: Cold users not allowed
DataSpl

In [2]:
raw_preds

,UserID,item_recommendations,item_similarity,recommender_name
0,1,"[79, 102, 24, 456, 504]","[2.4238963, 7.0842423, 3.720988, 15.40752, 1.8...",ItemKNNCFRecommender
1,8,"[31, 231, 161, 329, 191]","[17.841475, 22.730019, 5.355295, 32.019455, 5....",ItemKNNCFRecommender
2,9,"[52, 320, 22, 82, 367]","[7.45508, 15.457853, 16.431026, 55.0299, 2.797...",ItemKNNCFRecommender
3,14,"[24, 103, 23, 112, 95]","[8.622261, 20.879557, 3.1826096, 18.239925, 2....",ItemKNNCFRecommender
4,15,"[571, 240, 496, 856, 166]","[21.229681, 29.139444, 3.179411, 12.321476, 9....",ItemKNNCFRecommender
...,...,...,...,...
81,545,"[166, 256, 6, 217, 53]","[13.338856, -inf, 13.82879, 48.31776, 7.094194...",ItemKNNCFRecommender
82,556,"[68, 552, 124, 120, 336]","[0.9776668, 1.4022654, 0.53901714, 0.0, 0.0, 0...",ItemKNNCFRecommender
83,557,"[404, 95, 595, 364, 251]","[-inf, -inf, 1.4384379, 1.056989, -inf, 13.355...",ItemKNNCFRecommender
84,573,"[118, 221, 53, 356, 320]","[11.846922, -inf, 3.5545197, 28.096725, -inf, ...",ItemKNNCFRecommender
